In [1]:
import tensorflow.keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input, GlobalMaxPool1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os 

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
filename = '../dataset_text/miditokens.txt'
with open(filename) as f:
    miditokens = f.readlines()
    
midiTokensWithMeta = [tokens.strip().split(' ') for tokens in miditokens]

miditokens = []
for i in range(len(midiTokensWithMeta)):
    if 'timesig:4/4' in midiTokensWithMeta[i]:
        midiInfo = midiTokensWithMeta[i]
        
        if midiInfo[1].startswith('tempo'):
            midiInfo.remove(midiInfo[1])
        
        midiInfo.remove('timesig:4/4')
        
        miditokens.append(midiTokensWithMeta[i])
miditokens = miditokens[:100]

In [4]:
tokenizer = Tokenizer() # token -> int
tokenizer.fit_on_texts(miditokens)

In [5]:
#sample = miditokens[1]
#print(sample)
#print(tokenizer.texts_to_sequences([sample])[0]) # Example of token input -> int output
#print("Sõna sagedus kogu andmestikus", tokenizer.word_counts['end'])
#print("Sõna -> indeks teisendus", tokenizer.word_index['note:c4:v112'])
#print("Indeks -> sõna teisendus", tokenizer.index_word[55])
print(len(tokenizer.word_index))
#print(tokenizer.index_word)
#print(tokenizer.word_index)

249


In [6]:
# Turn all tokens to ints
midiTokensAsInt = tokenizer.texts_to_sequences(miditokens)
#midiTokensAsInt = np.array(midiTokensAsInt)

# Generator variant

In [7]:
## https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366
#
#def batchGenerator(trainData, VOCAB_SIZE, SEQ_LEN=30):
#    i = 0
#    while True:
#        yield loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i)
#        i += 1
#    
#            
#def loadDataBatch(trainData, VOCAB_SIZE, SEQ_LEN, i):
#    X = []
#    y = []
#    
#    # https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
#    
#    song = trainData[i]
#    for i in range(0, len(song) - SEQ_LEN, 1):
#        X.append(song[i:i + SEQ_LEN])
#        y.append(song[i + SEQ_LEN])
#            
#    input_len = len(X)
#    
#    # reshape the input into a format compatible with LSTM layers
#    X = np.reshape(X, (input_len, SEQ_LEN, 1))
#    
#    # normalize input
#    X = X / float(VOCAB_SIZE)
#    y = to_categorical(y, num_classes = VOCAB_SIZE)
#    
#    return (X, y)
#
#VOCAB_SIZE = len(tokenizer.word_index)
#SEQ_LEN = 20
#
#batchGen = batchGenerator(midiTokensAsInt, VOCAB_SIZE, SEQ_LEN)

In [8]:
#model = Sequential()
#model.add(LSTM(
#    256,
#    input_shape=(20, 1),
#    return_sequences=True
#))
#model.add(Dropout(0.3))
#model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(256))
#model.add(Dense(256))
#model.add(Dropout(0.3))
#model.add(Dense(VOCAB_SIZE))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#
#filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
#checkpoint = ModelCheckpoint(
#    filepath, monitor='loss', 
#    verbose=0,        
#    save_best_only=True,        
#    mode='min'
#)    
#callbacks_list = [checkpoint]     
##model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
#
#model.fit(batchGen, 
#          epochs=200,
#          callbacks=callbacks_list,
#          verbose=1)

# Non-generator variant

In [9]:
X = []
y = []

VOCAB_SIZE = len(tokenizer.word_index)
SEQ_LEN = 100

# https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

for song in midiTokensAsInt:
    for i in range(0, len(song) - SEQ_LEN, 1):
        X.append(song[i:i + SEQ_LEN])
        y.append(song[i + SEQ_LEN])

X_train = X
y_train = y

input_len = len(X_train)

# reshape the input into a format compatible with LSTM layers
X_train = np.reshape(X_train, (input_len, SEQ_LEN, 1))

# normalize input
X_train = X_train / float(VOCAB_SIZE)

y_train = to_categorical(y_train)

In [10]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE+1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "../weights/weights-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X_train, y_train, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
1898/1898 [==============================] - 131s 66ms/step - loss: 4.5352
Epoch 2/200
1898/1898 [==============================] - 124s 65ms/step - loss: 3.9878
Epoch 3/200
1898/1898 [==============================] - 124s 65ms/step - loss: 3.5446
Epoch 4/200
1898/1898 [==============================] - 125s 66ms/step - loss: 3.1954
Epoch 5/200
1898/1898 [==============================] - 125s 66ms/step - loss: 2.9275
Epoch 6/200
1898/1898 [==============================] - 124s 65ms/step - loss: 2.7369
Epoch 7/200
1898/1898 [==============================] - 124s 65ms/step - loss: 2.5856
Epoch 8/200
1898/1898 [==============================] - 124s 65ms/step - loss: 2.4574
Epoch 9/200
1898/1898 [==============================] - 124s 65ms/step - loss: 2.3474
Epoch 10/200
1898/1898 [==============================] - 124s 65ms/step - loss: 2.2555
Epoch 11/200
1898/1898 [==============================] - 124s 65ms/step - loss: 2.1837
Epoch 12/200
1898/1898 [=================

1898/1898 [==============================] - 124s 65ms/step - loss: 1.1201
Epoch 94/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.1139
Epoch 95/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.1154
Epoch 96/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.1197
Epoch 97/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.1065
Epoch 98/200
1898/1898 [==============================] - 125s 66ms/step - loss: 1.1006
Epoch 99/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.0961
Epoch 100/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.0985
Epoch 101/200
1898/1898 [==============================] - 124s 66ms/step - loss: 1.0919
Epoch 102/200
1898/1898 [==============================] - 125s 66ms/step - loss: 1.0868
Epoch 103/200
1898/1898 [==============================] - 125s 66ms/step - loss: 1.0869
Epoch 104/200
1898/1898 [================

KeyboardInterrupt: 

In [11]:
model.save("LSTM256.h5")

In [12]:
new_model = tf.keras.models.load_model('LSTM256.h5')

In [15]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(VOCAB_SIZE+1))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Load the weights to each node
model.load_weights('../weights/weights-178-0.9515-bigger.hdf5')

In [32]:
start = np.random.randint(0, len(X)-1)
intToNote = dict(map(reversed, tokenizer.word_index.items()))

nrOfSong = 47

pattern = midiTokensAsInt[nrOfSong][:100]
#pattern = [i for [i] in pattern]
prediction_output = []
for tokenInt in pattern:
    prediction_output.append(intToNote[tokenInt])

# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(VOCAB_SIZE)
    
    prediction = model.predict(prediction_input, verbose=0)
    
    index = np.argmax(prediction)
    result = intToNote[index]
    prediction_output.append(result)
    
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [33]:
print("Whole song:")
text2midi(miditokens[nrOfSong]).show("midi")
print("Input to network:")
text2midi(miditokens[nrOfSong][:100]).show("midi")

Whole song:


Input to network:


In [34]:
#print(prediction_output)
genmidistream = text2midi(prediction_output)
genmidistream.show("midi")

In [22]:
#fp = genmidistream.write('midi', fp='wow.mid')

In [18]:
from music21 import *

def text2midi(tokens):
    s = stream.Stream()
    
    currentVelocity = 0

    currentOffset = 0
    currentToken = 0

    for token in tokens:

        splitToken = token.split(":")

        if token.startswith("tempo"):
            s.append(tempo.MetronomeMark(number=float(splitToken[1])))

        if token.startswith("timesig"):
            s.append(meter.TimeSignature(splitToken[1]))
            
        if token.startswith("velocity"):
            currentVelocity = int(splitToken[1])

        if token.startswith("note") and not token.lower().endswith("off"):
            noteDuration = 0
            noteName = splitToken[1]

            for element in tokens[currentToken+1:]:
                splitToken2 = element.split(":")
                if (element.startswith("wait")):
                    noteDuration += float(splitToken2[1])
                if (element.startswith("note") and element.lower().endswith("off")):
                    if (noteName == splitToken2[1]):
                        newNote = note.Note(nameWithOctave=splitToken[1],  
                               quarterLength=float(noteDuration))
                        newNote.volume.velocity = currentVelocity
                        s.insert(currentOffset, newNote)
                        break

        if token.startswith("wait"):
            currentOffset += float(splitToken[1]) 

        currentToken += 1

    return s

#text2midi(midiTokens).show("midi")